<a href="https://colab.research.google.com/github/marionwenger/DLColabNotebooks/blob/main/MRI_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# '/content/drive/MyDrive/DLforMRI'
# save files on drive or locally, they are deleted in colab after 12 hours at the latest

Mounted at /content/drive


In [13]:
!pip install split-folders

In [14]:
# copy from Nicola's resnet18-model notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy # A module that provides functions for creating copies of objects, useful for avoiding unintended modifications to variables.
import os # A module that provides a way to interact with the operating system, allowing for tasks such as file and directory manipulation.
import torch
from PIL import Image # A module from the Python Imaging Library (PIL) that provides functionality for opening, manipulating, and saving various image file formats.
from torch.utils.data import Dataset # A PyTorch class that represents a dataset and provides an interface for accessing and processing the data during training.
import torchvision
import torchvision.transforms.v2 as transforms # A module from the torchvision library that provides common image transformations, such as resizing, cropping, and normalization.
# v2 is recommended and gets udpates
from torch.utils.data import random_split # A function from PyTorch that allows for randomly splitting a dataset into training and validation subsets.
from torch.optim.lr_scheduler import ReduceLROnPlateau # A PyTorch scheduler that adjusts the learning rate during training based on a specified metric, reducing it when the metric plateaus.
import torch.nn as nn # A module in PyTorch that provides classes for defining and building neural networks.
from torchvision import utils # A module from torchvision that contains utility functions for working with images, such as saving and visualizing them.
from torchvision.datasets import ImageFolder
import splitfolders
from torchsummary import summary
import torch.nn.functional as F
import pathlib


In [15]:
os.listdir(r'/content/drive/MyDrive/DLforMRI/Brain Tumor Data')

# ['MRI files', 'metadata.csv', '.DS_Store']

['MRI files', 'metadata.csv', '.DS_Store']

In [16]:
# Dataset Path
data_path_original = pathlib.Path(r'/content/drive/MyDrive/DLforMRI/Brain Tumor Data')

# Splitting dataset to train_set, val_set and test_set
splitfolders.ratio(data_path_original, output='MRI splitted', seed=1404, ratio=(0.6, 0.3, 0.1), move=True)
# set move=True if you want to move the files instead of copying.

# New dataset path
data_path_splitted = pathlib.Path(r'/content/drive/MyDrive/DLforMRI/Brain Tumor Data/MRI splitted')

os.listdir(r'/content/drive/MyDrive/DLforMRI/Brain Tumor Data/MRI splitted')

ValueError: The input data is not in a right format. Within your folder "/content/drive/MyDrive/DLforMRI/Brain Tumor Data/MRI files" there are no directories. Consult the documentation how to the folder structure should look like.

In [ ]:
meta_data = pd.read_csv(r'/content/drive/MyDrive/DLforMRI/Brain Tumor Data/metadata.csv')
meta_data.head()


In [ ]:
# define transformation
transform = transforms.Compose(
    [
        transforms.Resize((256,256)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(30),
        transforms.ToTensor() #,
        #transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225]) # Z-Transformation für die 3 Input-Channels
        # denke brauche keine Normalisation hier und falls doch, wäre es besser, die mean udn std zu kennen...
        ]
)

In [ ]:
# Transform the train and validation data in their respective folders
train_set = torchvision.datasets.ImageFolder(data_path_splitted.joinpath("train"), transform=transform)
train_set.transform
val_set = torchvision.datasets.ImageFolder(data_path_splitted.joinpath("val"), transform=transform)
val_set.transform



In [ ]:
# import and load train, validation
batch_size = 16
# https://www.mdpi.com/2079-9292/12/4/964
# The research study concluded that batch sizes between 2 and 32 produced good results and added that small batch sizes are more robust than high batch sizes.

train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 2)
val_loader = torch.utils.data.DataLoader(val_set, batch_size = batch_size, shuffle = True, num_workers = 2)

In [ ]:
# print shape for Training data and Validation data
for key, value in {'Training Data': train_loader, "Validation Data": val_loader}.items():
    for X, Y in value:
        print(f"{key}:")
        print(f"Shape of X : {X.shape}")
        print(f"Shape of Y : {Y.shape}")
        print(f"Type  of Y : {Y.dtype}")
        break

In [ ]:
efficientnet_model = hub.KerasLayer("https://www.kaggle.com/models/google/bit/frameworks/TensorFlow2/variations/m-r101x1/versions/1",
                                 trainable=False,
                                 input_shape=(28, 28, 3),
                                 name='efficientnet')

X_train_tensor_rgb = tf.image.grayscale_to_rgb(X_train_tensor)
X_test_tensor_rgb = tf.image.grayscale_to_rgb(X_test_tensor)

model_5 = Sequential([
    efficientnet_model,
    Dense(10, activation='softmax')
])

model_5.compile(loss=CategoricalCrossentropy(),
                optimizer=Adam(),
                metrics=['accuracy'])

In [ ]:
history_5 = model_5.fit(x=X_train_tensor_rgb,
                        y=y_train_tensor,
                        steps_per_epoch=100,
                        epochs=15,
                        validation_data=(X_test_tensor_rgb, y_test_tensor),
                        callbacks=[scheduler],
                        verbose=0)

In [ ]:
best_lr_model_5 = get_best_learning_rate(history_5)
print(best_lr_model_5)

In [ ]:
odel_6 = clone_model(model_5)

model_6.compile(loss=CategoricalCrossentropy(),
                optimizer=Adam(best_lr_model_5),
                metrics=['accuracy'])

In [ ]:
tensorboard_callback_efficientnet_model = create_tensorboard_callbacks('efficientnet_model')

history_6 = model_6.fit(x=X_train_tensor_rgb,
                        y=y_train_tensor,
                        epochs=10,
                        validation_data=(X_test_tensor_rgb, y_test_tensor),
                        batch_size=32,
                        callbacks=[tensorboard_callback_efficientnet_model, early_stopping])

In [ ]:
model_6.summary()

In [ ]:
model_6.evaluate(x=X_test_tensor_rgb, y=y_test_tensor, steps=len(X_test_tensor))

In [ ]:
plot_result_model(history_6)